In [1]:
import glob
import pandas as pd

/home/reza/projects/medium-scraper/.venv/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def clean_claps(x):
    if x[-1] == 'K':
        return int(float(x[:-1]) * 1000)
    else:
        return int(x)

In [3]:
def fix_title(title, url):
    tokens = url.split('/')[-1].split('-')[:-1]
    title = ' '.join(tokens).capitalize()
    return title

In [4]:
def fix_author(author, url):
    return url.split('/')[-2].replace('_', ' ').replace('-', ' ').strip().title()

In [5]:
filenames = glob.glob("../data/raw/*.csv")
frames = [pd.read_csv(f) for f in filenames]
medium = pd.concat(frames)

In [6]:
# Make column names lowercase
old = medium.columns.tolist()
new = [col.lower() for col in old]
colmap = dict(zip(old, new))
medium.rename(columns=colmap, inplace=True)

In [7]:
medium.head()

,title,subtitle,image,author,publication,year,month,day,tag,reading_time,claps,comment,url,author_url
0,Our Ongoing Analytics Project with the NFL: Wi...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,1,data-science,3,0,0,https://medium.com/opex-analytics/our-ongoing-...,https://medium.com/@OpexAnalytics?source=tag_a...
1,Benefits of Map Portals for Your Analytics Pro...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,3,data-science,1,0,0,https://medium.com/opex-analytics/benefits-of-...,https://medium.com/@OpexAnalytics?source=tag_a...
2,Data Inference: Drinking With the Dude,A guide to detecting biases in your data.,1,Tapad Engineering,Tapad Engineering,2014,4,8,data-science,7,1,0,https://engineering.tapad.com/data-inference-d...,https://engineering.tapad.com/@tapadeng?source...
3,WSJ Article on the Importance of Analytics to ...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,11,data-science,1,0,0,https://medium.com/opex-analytics/wsj-article-...,https://medium.com/@OpexAnalytics?source=tag_a...
4,Sample Custom Optimization Scheduling Problem,by Prateek Rastogi,0,Opex Analytics,The Opex Analytics Blog,2014,4,14,data-science,1,0,0,https://medium.com/opex-analytics/sample-custo...,https://medium.com/@OpexAnalytics?source=tag_a...


In [8]:
medium.dtypes

title           object
subtitle        object
image            int64
author          object
publication     object
year             int64
month            int64
day              int64
tag             object
reading_time     int64
claps           object
comment          int64
url             object
author_url      object
dtype: object

In [9]:
print(f'Total entries: {medium.shape[0]}')

Total entries: 77432


In [10]:
# Remove duplicates
medium.drop_duplicates(inplace=True, ignore_index=True)

In [11]:
print(f'Deduped entries: {medium.shape[0]}')

Deduped entries: 77309


In [12]:
# Clean claps
medium['claps'] = medium['claps'].apply(clean_claps)

In [13]:
# Remove comments
medium = medium[medium['comment']==0].reset_index(drop=True)

In [14]:
print(f'Non-comment entries: {medium.shape[0]}')

Non-comment entries: 74586


In [15]:
# Clean up URLs
medium['url'] = medium['url'].str.split('?', expand=True).iloc[:, 0]
medium['author_url'] = medium['author_url'].str.split('?', expand=True).iloc[:, 0]

In [16]:
# Fix null titles
medium['fake_title'] = 0
medium.loc[medium['title'].isnull(), 'fake_title'] = 1
medium.loc[medium['fake_title'] == 1, 'title'] = medium.apply(lambda x: fix_title(x['title'], x['url']), axis=1)

In [17]:
# Fix null authors
medium['fake_author'] = 0
medium.loc[medium['author'].isnull(), 'fake_author'] = 1
medium.loc[medium['fake_author'] == 1, 'author'] = medium.apply(lambda x: fix_author(x['author'], x['url']), axis=1)

In [18]:
medium.isnull().sum()

title               0
subtitle        26607
image               0
author              0
publication     37143
year                0
month               0
day                 0
tag                 0
reading_time        0
claps               0
comment             0
url                 0
author_url         48
fake_title          0
fake_author         0
dtype: int64

In [19]:
# Add article date
medium['date'] = pd.to_datetime(medium[['year', 'month', 'day']])

In [20]:
# Save clean data
medium.to_csv('../data/processed/medium.csv', index=False)

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/home/reza/projects/medium-scraper/.venv/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def clean_claps(x):
    if x[-1] == 'K':
        return int(float(x[:-1]) * 1000)
    else:
        return int(x)

In [3]:
def fix_title(title, url):
    tokens = url.split('/')[-1].split('-')[:-1]
    title = ' '.join(tokens).capitalize()
    return title

In [4]:
def fix_author(author, url):
    return url.split('/')[-2].replace('_', ' ').replace('-', ' ').strip().title()

In [5]:
filenames = glob.glob("../data/raw/*.csv")
frames = [pd.read_csv(f) for f in filenames]
medium = pd.concat(frames)

In [6]:
# Make column names lowercase
old = medium.columns.tolist()
new = [col.lower() for col in old]
colmap = dict(zip(old, new))
medium.rename(columns=colmap, inplace=True)

In [7]:
medium.head()

,title,subtitle,image,author,publication,year,month,day,tag,reading_time,claps,comment,url,author_url
0,Our Ongoing Analytics Project with the NFL: Wi...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,1,data-science,3,0,0,https://medium.com/opex-analytics/our-ongoing-...,https://medium.com/@OpexAnalytics?source=tag_a...
1,Benefits of Map Portals for Your Analytics Pro...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,3,data-science,1,0,0,https://medium.com/opex-analytics/benefits-of-...,https://medium.com/@OpexAnalytics?source=tag_a...
2,Data Inference: Drinking With the Dude,A guide to detecting biases in your data.,1,Tapad Engineering,Tapad Engineering,2014,4,8,data-science,7,1,0,https://engineering.tapad.com/data-inference-d...,https://engineering.tapad.com/@tapadeng?source...
3,WSJ Article on the Importance of Analytics to ...,by Michael Watson,0,Opex Analytics,The Opex Analytics Blog,2014,4,11,data-science,1,0,0,https://medium.com/opex-analytics/wsj-article-...,https://medium.com/@OpexAnalytics?source=tag_a...
4,Sample Custom Optimization Scheduling Problem,by Prateek Rastogi,0,Opex Analytics,The Opex Analytics Blog,2014,4,14,data-science,1,0,0,https://medium.com/opex-analytics/sample-custo...,https://medium.com/@OpexAnalytics?source=tag_a...


In [8]:
medium.dtypes

title           object
subtitle        object
image            int64
author          object
publication     object
year             int64
month            int64
day              int64
tag             object
reading_time     int64
claps           object
comment          int64
url             object
author_url      object
dtype: object

In [9]:
print(f'Total entries: {medium.shape[0]}')

Total entries: 77432


In [10]:
# Remove duplicates
medium.drop_duplicates(inplace=True, ignore_index=True)

In [11]:
print(f'Deduped entries: {medium.shape[0]}')

Deduped entries: 77309


In [12]:
# Clean claps
medium['claps'] = medium['claps'].apply(clean_claps)

In [13]:
# Remove comments
medium = medium[medium['comment']==0].reset_index(drop=True)

In [14]:
print(f'Non-comment entries: {medium.shape[0]}')

Non-comment entries: 74586


In [15]:
# Clean up URLs
medium['url'] = medium['url'].str.split('?', expand=True).iloc[:, 0]
medium['author_url'] = medium['author_url'].str.split('?', expand=True).iloc[:, 0]

In [16]:
# Fix null titles
medium['fake_title'] = 0
medium.loc[medium['title'].isnull(), 'fake_title'] = 1
medium.loc[medium['fake_title'] == 1, 'title'] = medium.apply(lambda x: fix_title(x['title'], x['url']), axis=1)

In [17]:
# Fix null authors
medium['fake_author'] = 0
medium.loc[medium['author'].isnull(), 'fake_author'] = 1
medium.loc[medium['fake_author'] == 1, 'author'] = medium.apply(lambda x: fix_author(x['author'], x['url']), axis=1)

In [18]:
medium.isnull().sum()

title               0
subtitle        26607
image               0
author              0
publication     37143
year                0
month               0
day                 0
tag                 0
reading_time        0
claps               0
comment             0
url                 0
author_url         48
fake_title          0
fake_author         0
dtype: int64

In [19]:
# Add article date
medium['date'] = pd.to_datetime(medium[['year', 'month', 'day']])

In [20]:
# Save clean data
medium.to_csv('../data/processed/medium.csv', index=False)